In [ ]:
"""
TO DO:

- add wind power production -> Andre, Joschi
- add distingtion betweent taks priority -> Tanver, Lorenz
- data reasearch: price per battery capacity (mx a+mx), solar panel, small vertical wind turbine -> Emanuele
- plots
- summer/winter
- (co2 equivalent)
- (monetization)
- (sensitivity analysis)
- (dynamic importance of tasks)

"""

<h2>Imports</h2>

In [62]:
# Import libraries

import simpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from util import Battery, read_battery_process, solar_charge_process, compute_tasks_process

<h2>Constraints</h2>

In [ ]:
# Set constraints

minimal_reliability = 0.99

<h2>Data prep</h2>

In [3]:
# Load and prepare data from csv-files
START_DATE = pd.to_datetime("2021-06-01 00:00:00")
END_DATE = pd.to_datetime("2021-06-08 00:00:00")
INDEX = pd.date_range(START_DATE, END_DATE, freq="s")  # +timedelta(minutes=1)


# Load data between START_DATE and END_DATE
with open("data/weather_berlin_2021-06.csv", "r") as csvfile:
    weather_df = pd.read_csv(csvfile, parse_dates=True, index_col=0)[START_DATE:END_DATE]
with open("data/low_prio_tasks.csv", "r") as csvfile:
    tasks_df = pd.read_csv(csvfile, parse_dates=True, index_col=0)[START_DATE:END_DATE]
with open("data/high_prio_tasks.csv", "r") as csvfile:
    hptasks_df = pd.read_csv(csvfile, parse_dates=True, index_col=0)[START_DATE:END_DATE]

<h2>Scenario and costs</h2>

In [65]:
# Set researched values for costs etc. depending on the scenario

cost_per_capacity = 1000
cost_per_solar = 500

BATTERY_CAPACITIES = list(np.linspace(500, 6000, 5))     # Wh
SOLAR_AREAS = list(np.linspace(0.1, 3, 5))          # m^2

<h2>Simulation Function</h2>

In [37]:
def simulation(panel, capacity):
    """
    Runs simulation for a specific setup.
    """

    # Calculate set-up cost
    cost = cost_per_capacity*capacity + cost_per_solar*panel

    # Initialize Battery
    battery = Battery(capacity=capacity, soc=capacity)
    battery_socs = []
    task_stats = []

    # Run simulation
    env = simpy.Environment()
    env.process(solar_charge_process(env, battery, solar_area=panel, solar_values=weather_df["solar"]))
    env.process(compute_tasks_process(env, battery, start_date=START_DATE, task_arrival_times=tasks_df.index, task_stats=task_stats))
    env.process(read_battery_process(env, battery, battery_socs))
    env.run(until=len(INDEX))

    # Calculate stats
    rate = check_stats(task_stats)
    print("____config done____")

    return cost, rate



def check_stats(stats):
    """
    Calculates successrate from stats.
    """

    success_count = 0

    for task in stats:
        if task[1]:
            success_count += 1

    success_rate = success_count/len(stats)

    return success_rate

<h2>Main</h2>

In [66]:
results = np.zeros((len(BATTERY_CAPACITIES), len(SOLAR_AREAS), 2))
t1= time.time()
for count_p, panel in enumerate(SOLAR_AREAS):
    for count_b, capacity in enumerate(BATTERY_CAPACITIES):

        cost, rate = simulation(panel, capacity)

        results[count_b, count_p, 0] = cost
        results[count_b, count_p, 1] = rate
        if count_b==len(BATTERY_CAPACITIES):
            print('finished column')
print(time.time()-t1)
print(results)

____config done____
____config done____
____config done____
____config done____
____config done____
____config done____
____config done____
____config done____
____config done____
____config done____
____config done____
____config done____
____config done____
____config done____
____config done____
____config done____
____config done____
____config done____
____config done____
____config done____
____config done____
____config done____
____config done____
____config done____
____config done____
64.28487992286682
[[[5.00050000e+05 5.68780955e-02]
  [5.00412500e+05 3.85764499e-01]
  [5.00775000e+05 7.50758907e-01]
  [5.01137500e+05 8.80731746e-01]
  [5.01500000e+05 9.21233424e-01]]

 [[1.87505000e+06 1.75107845e-01]
  [1.87541250e+06 5.25563189e-01]
  [1.87577500e+06 8.89598977e-01]
  [1.87613750e+06 1.00000000e+00]
  [1.87650000e+06 1.00000000e+00]]

 [[3.25005000e+06 2.92698514e-01]
  [3.25041250e+06 6.59370506e-01]
  [3.25077500e+06 1.00000000e+00]
  [3.25113750e+06 1.00000000e+00]
  

<h2>Results</h2>

In [55]:
print(results)

[[[5.00050000e+05 5.68780955e-02]
  [5.00100000e+05 8.71544975e-02]
  [5.00150000e+05 1.24141237e-01]
  [5.00200000e+05 1.72711296e-01]
  [5.00250000e+05 2.28790542e-01]
  [5.00300000e+05 2.76561751e-01]
  [5.00350000e+05 3.22335836e-01]
  [5.00400000e+05 3.71544975e-01]
  [5.00450000e+05 4.25627097e-01]
  [5.00500000e+05 4.80348298e-01]
  [5.00550000e+05 5.32832721e-01]
  [5.00600000e+05 5.89950471e-01]
  [5.00650000e+05 6.39958460e-01]
  [5.00700000e+05 6.89487139e-01]
  [5.00750000e+05 7.33343985e-01]
  [5.00800000e+05 7.63460617e-01]
  [5.00850000e+05 7.87825531e-01]
  [0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00]
  [0.00000000e